Use gridsearch to see if there are more optimal lightGBM parameters we should be using.

We'll use the features that a have already been generated for our current best experiment (third sentinel + land cover features, trained with folds). Compare the results to that [best experiment](https://docs.google.com/presentation/d/1zWrSMSivxylx_iH_aOapJfyziRsDuyuXOELduOn6x3c/edit#slide=id.g278eb39bdd6_0_43)

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import yaml

from cloudpathlib import AnyPath
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import GridSearchCV

from cyano.data.utils import add_unique_identifier
from cyano.experiment.experiment import ExperimentConfig
from cyano.pipeline import CyanoModelPipeline
from cyano.settings import RANDOM_STATE

In [12]:
tmp_dir = AnyPath("tmp_dir")
tmp_dir.mkdir(exist_ok=True)

In [13]:
experiment_dir = AnyPath(
    "s3://drivendata-competition-nasa-cyanobacteria/experiments/results/third_sentinel_with_folds"
)

In [14]:
train_features = pd.read_csv(experiment_dir / "features_train.csv", index_col=0)
train_features.head()

,AOT_mean,AOT_min,AOT_max,AOT_range,B01_mean,B01_min,B01_max,B01_range,B02_mean,B02_min,...,WVP_min,WVP_max,WVP_range,NDVI_B04,NDVI_B05,NDVI_B06,NDVI_B07,month,days_before_sample,land_cover
sample_id,,,,,,,,,,,,,,,,,,,,,
0001cfa683171fe80161cdbe6a090c94,102.0,102.0,102.0,0.0,319.843750,144.0,1083.0,939.0,441.956349,146.0,...,277.0,1413.0,1136.0,0.581754,0.418335,0.114064,0.033551,2.0,15.0,11
0001cfa683171fe80161cdbe6a090c94,132.0,132.0,132.0,0.0,435.984375,229.0,1055.0,826.0,503.288549,154.0,...,167.0,828.0,661.0,0.574375,0.418914,0.120972,0.042572,3.0,8.0,11
0001cfa683171fe80161cdbe6a090c94,204.0,204.0,204.0,0.0,902.640625,703.0,1661.0,958.0,938.553288,672.0,...,931.0,931.0,0.0,0.455419,0.329302,0.097910,0.027511,3.0,5.0,11
0001cfa683171fe80161cdbe6a090c94,102.0,102.0,102.0,0.0,518.484375,316.0,1554.0,1238.0,618.096939,277.0,...,574.0,1436.0,862.0,0.548216,0.391690,0.115218,0.032818,2.0,13.0,11
0001cfa683171fe80161cdbe6a090c94,102.0,102.0,102.0,0.0,318.218750,104.0,1314.0,1210.0,485.587302,162.0,...,340.0,1114.0,774.0,0.582335,0.425137,0.129981,0.039196,2.0,10.0,11


In [17]:
# Load train labels
train = pd.read_csv(
    AnyPath(
        "s3://drivendata-competition-nasa-cyanobacteria/experiments/splits/competition/train.csv"
    )
)
train = add_unique_identifier(train)
train.shape

(17060, 10)

In [18]:
train.head(2)

,uid,data_provider,region,latitude,longitude,date,density_cells_per_ml,severity,distance_to_water_m,log_density
sample_id,,,,,,,,,,
d7ebbce63c7d1498cc627a1e77f6061c,aabm,Indiana State Department of Health,midwest,39.080319,-86.430867,2018-05-14,585.0,1,0.0,6.373320
0856c3740614b5ee606f82d6c3a215a0,aacd,N.C. Division of Water Resources N.C. Departme...,south,35.875083,-78.878434,2020-11-19,290.0,1,514.0,5.673323


In [20]:
train.loc[train_features.index].log_density

sample_id
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
                                      ...    
fff76b0c751a22eda5a33f3f0d7fda98    10.338188
fff948d10c4ef9e03fddb358140b2755     9.275660
fff948d10c4ef9e03fddb358140b2755     9.275660
fff948d10c4ef9e03fddb358140b2755     9.275660
fff948d10c4ef9e03fddb358140b2755     9.275660
Name: log_density, Length: 43409, dtype: float64

Try params from each of the winners

In [52]:
param_grid = {
    'max_depth': [-1, 8],
    # 'num_leaves': [31],
    # 'learning_rate': [0.005, 0.1],
    # 'bagging_fraction': [0.3, 1.0],
    # 'feature_fraction': [0.3, 1.0],
    'min_split_gain': [0.0, 0.1],
    # 'n_estimators': [1000, 100000, 470], # same as num_boost_round
}

In [84]:
lgb_model = lgb.LGBMRegressor(objective='regression', metric='rmse')

In [85]:
grid_search = GridSearchCV(
    estimator = lgb_model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='r2'
)

In [86]:
grid_search.fit(
    train_features,
    train.loc[train_features.index].log_density
)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15445
[LightGBM] [Info] Number of data points in the train set: 43409, number of used features: 67
[LightGBM] [Info] Start training from score 10.858860


GridSearchCV(cv=5,
             estimator=LGBMRegressor(metric='rmse', objective='regression'),
             n_jobs=-1,
             param_grid={'max_depth': [-1, 8], 'min_split_gain': [0.0, 0.1]},
             scoring='r2')

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.061451 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15436
[LightGBM] [Info] Number of data points in the train set: 34727, number of used features: 67
[LightGBM] [Info] Start training from score 10.878464
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032570 seconds.
You can set `force_col_wis

If we don't specify 'scoring' in `grid_search`, I think score is the `lgb_model.score` method, which is R^2. Unclear whether specifying `metric="rmse"` makes the score that is returned RMSE.

In [87]:
# with scoring specified in grid_search
pd.DataFrame(grid_search.cv_results_).sort_values(by='mean_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_split_gain,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,4.611209,1.878887,0.067850,0.022124,8,0.1,"{'max_depth': 8, 'min_split_gain': 0.1}",0.527846,0.490092,0.533387,0.521765,0.514624,0.517542,0.015080,3
2,12.263181,0.500616,0.039015,0.001223,8,0.0,"{'max_depth': 8, 'min_split_gain': 0.0}",0.527846,0.490092,0.533387,0.521765,0.514624,0.517542,0.015080,3
0,24.913565,2.236077,0.071177,0.018261,-1,0.0,"{'max_depth': -1, 'min_split_gain': 0.0}",0.533259,0.495535,0.535176,0.523156,0.517355,0.520896,0.014266,2
1,20.165612,4.943512,0.052979,0.015319,-1,0.1,"{'max_depth': -1, 'min_split_gain': 0.1}",0.533259,0.495535,0.535176,0.523156,0.517831,0.520991,0.014244,1


In [62]:
# without scoring specified in grid_search
pd.DataFrame(grid_search.cv_results_).sort_values(by='mean_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_split_gain,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,2.269762,0.703575,0.025750,0.001052,8,0.1,"{'max_depth': 8, 'min_split_gain': 0.1}",0.527846,0.490092,0.533387,0.521765,0.514624,0.517542,0.015080,3
2,3.715456,0.167349,0.029753,0.006178,8,0.0,"{'max_depth': 8, 'min_split_gain': 0.0}",0.527846,0.490092,0.533387,0.521765,0.514624,0.517542,0.015080,3
0,4.342626,0.529544,0.030479,0.005983,-1,0.0,"{'max_depth': -1, 'min_split_gain': 0.0}",0.533259,0.495535,0.535176,0.523156,0.517355,0.520896,0.014266,2
1,4.221056,0.384248,0.026779,0.001072,-1,0.1,"{'max_depth': -1, 'min_split_gain': 0.1}",0.533259,0.495535,0.535176,0.523156,0.517831,0.520991,0.014244,1


In [74]:
lgb_model.score

<bound method RegressorMixin.score of LGBMRegressor(eval_metric='rmse', objective='regression')>

In [73]:
?lgb_model.score

Signature: lgb_model.score(X, y, sample_weight=None)
Docstring:
Return the coefficient of determination of the prediction.

The coefficient of determination :math:`R^2` is defined as
:math:`(1 - \frac{u}{v})`, where :math:`u` is the residual
sum of squares ``((y_true - y_pred)** 2).sum()`` and :math:`v`
is the total sum of squares ``((y_true - y_true.mean()) ** 2).sum()``.
The best possible score is 1.0 and it can be negative (because the
model can be arbitrarily worse). A constant model that always predicts
the expected value of `y`, disregarding the input features, would get
a :math:`R^2` score of 0.0.

Parameters
----------
X : array-like of shape (n_samples, n_features)
    Test samples. For some estimators this may be a precomputed
    kernel matrix or a list of generic objects instead with shape
    ``(n_samples, n_samples_fitted)``, where ``n_samples_fitted``
    is the number of samples used in the fitting for the estimator.

y : array-like of shape (n_samples,) or (n_samples, 

In [66]:
grid_search.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__boosting_type': 'gbdt',
 'estimator__class_weight': None,
 'estimator__colsample_bytree': 1.0,
 'estimator__importance_type': 'split',
 'estimator__learning_rate': 0.1,
 'estimator__max_depth': -1,
 'estimator__min_child_samples': 20,
 'estimator__min_child_weight': 0.001,
 'estimator__min_split_gain': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__num_leaves': 31,
 'estimator__objective': 'regression',
 'estimator__random_state': None,
 'estimator__reg_alpha': 0.0,
 'estimator__reg_lambda': 0.0,
 'estimator__subsample': 1.0,
 'estimator__subsample_for_bin': 200000,
 'estimator__subsample_freq': 0,
 'estimator__eval_metric': 'rmse',
 'estimator': LGBMRegressor(eval_metric='rmse', objective='regression'),
 'n_jobs': -1,
 'param_grid': {'max_depth': [-1, 8], 'min_split_gain': [0.0, 0.1]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

In [64]:
grid_search.best_estimator_.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.1,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': 'regression',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'eval_metric': 'rmse'}

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026264 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15436
[LightGBM] [Info] Number of data points in the train set: 34727, number of used features: 67
[LightGBM] [Info] Start training from score 10.878464
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013548 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15433
[LightGBM] [Info] Number of data points in the train set: 34728, number of used features: 67
[LightGBM] [Info] Start training from score 10.861573
[LightGBM] [Warning] Accuracy may be bad since

In [21]:
lgb_data = lgb.Dataset(
    train_features,
    train.loc[train_features.index].log_density,
)